<a href="https://colab.research.google.com/github/TheBlock98/Thesis/blob/main/LSTM_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We want to build a LSTM model to predict future price  
(HLC3) of BTC using dataSetIMRCleaned.csv like dataSet.
First of all  we want to predict HLC3 to one step, in the second phase we want build multiOutput model to
predict HLC3 to:  1 steps in this phase. We implement using pyTorch.

We want use Boruta to do PCA and select important features.


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score


In [12]:
config = {
    "data": {
        "window_size": 200,
        "train_split_size": 0.70,
        "test_split_size": 0.20,
        "valid_split_size": 0.10,

    },

    "model": {
        "input_size": 62,  # since we are only using 62 feature, close price
        "num_lstm_layers": 1,
        "lstm_size": 64,
        "dropout": 0.8,
        "num_class": 1,
        "model_path": "/content/drive/MyDrive/Colab Notebooks",
        "model_name": "LSTMV0.txt",
        "save_model": True,
    },
    "training": {
        #"device": "cpu",  # "cuda" or "cpu"
        "batch_size": 64,
        "num_epochs": 100,
        "learning_rate": 0.01, # 1e-3
        "loss_function": nn.MSELoss(),
        "optimizer": torch.optim.Adam,
        "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau,
        "scheduler_patience": 3,
        "scheduler_factor": 0.5,
        "scheduler_min_lr": 1e-6,
        "early_stopping": True,
        "early_stopping_patience": 20,



      },
}

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
dataSet = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataSetIMRNormalized.csv")
hlc3 = dataSet['hlc3']
dataSet.drop(labels=['hlc3'], axis=1, inplace=True)
dataSet.insert(0, 'hlc3', hlc3)

# Dataloading (sliding windows)
config["model"]["input_size"] = dataSet.shape[1] # with 1 have number of features


In [15]:
def sliding_windows(df, windowSize):
  x = []
  y = []
  for i in range(len(df) - windowSize):
    # Convert DataFrame slice to values and append to x
    _x = df.iloc[i:(i + windowSize)].values
    # Append the hlc3 value at position i + windowSize as the target
    _y = df.iloc[i + windowSize]["hlc3"]
    x.append(_x)
    y.append(_y)

  return np.array(x), np.array(y)


In [16]:
x, y = sliding_windows(dataSet, config["data"]["window_size"])
train_size = int(len(y) * config["data"]["train_split_size"])
val_size = int(len(y) * config["data"]["valid_split_size"])
test_size = int(len(y) * config["data"]["test_split_size"])

# transform from np.array  x and y to tensors
dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

print(dataX.size())
# split seqences into train and test
trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))
valX = Variable(torch.Tensor(np.array(x[train_size:train_size + val_size])))
valY = Variable(torch.Tensor(np.array(y[train_size:train_size + val_size])))
print(valX.size())
testX = Variable(torch.Tensor(np.array(x[train_size + val_size:])))
testY = Variable(torch.Tensor(np.array(y[train_size + val_size:])))


torch.Size([50144, 200, 62])
torch.Size([5014, 200, 62])


In [17]:
class LSTM(nn.Module):
  def __init__(self, input_size, num_lstm_layers, lstm_size, dropout, num_class):
    super(LSTM, self).__init__()
    self.input_size = 62
    self.num_lstm_layers = num_lstm_layers
    self.lstm_size = lstm_size
    self.dropout = dropout
    self.num_class = num_class

    # Fully connected layer as input layer
    self.fc_input = nn.Linear(62, lstm_size)


    # Define the LSTM layers
    self.lstm = nn.LSTM(input_size=config["model"]["lstm_size"],
                        hidden_size=config["model"]["lstm_size"],
                        num_layers=config["model"]["num_lstm_layers"],
                        dropout=dropout,
                        batch_first=True)
    # Define the Dropout layer
    # self.dropout_layer = nn.Dropout(dropout)

    # Define the output fully connected layer
    # Define the output fully connected layer
    self.fc_output = nn.Linear(lstm_size, config["model"]["num_class"])


  def forward(self, x):

    # Pass data through the input fully connected layer
    x = self.fc_input(x)
    x = F.relu(x)

    # Initialize hidden and cell states with zeros
    h_0 = Variable(torch.zeros(self.num_lstm_layers, x.size(0), self.lstm_size))
    c_0 = Variable(torch.zeros(self.num_lstm_layers, x.size(0), self.lstm_size))

      # Pass through LSTM layers
    out = x
    for i in range(self.num_lstm_layers):
      out, (h_0, c_0) = self.lstm(out, (h_0.detach(), c_0.detach()))
      out = F.relu(out)
      if self.num_lstm_layers > 1:
        out = self.dropout_layer(out)

    out = out[:, -1, :]
    # Output layer
    output = self.fc_output(out)  # Sele
    return output

In [18]:
model = LSTM(input_size=config["model"]["input_size"],
             num_lstm_layers=config["model"]["num_lstm_layers"],
             lstm_size=config["model"]["lstm_size"],
             dropout=config["model"]["dropout"],
             num_class=1)

optimizer = torch.optim.Adam(model.parameters(), lr=config["training"]["learning_rate"])
# optimizer = optim.RMSprop(model.parameters(), lr=config["training"]["learning_rate"], weight_decay=weight_decay)
# optimizer = optim.AdamW(model.parameters(), lr=config["training"]["learning_rate"], weight_decay=weight_decay)
#optimizer = torch.optim.SGD(model.parameters(), lr=config["training"]["learning_rate"])
criterion = torch.nn.MSELoss()    # mean-squared error for regression
scheduler = config["training"]["scheduler"](optimizer, patience=config["training"]["scheduler_patience"],
                                            factor=config["training"]["scheduler_factor"],
                                            min_lr=config["training"]["scheduler_min_lr"])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [25]:
best_val_loss = float('inf')
patience_counter = 0

class EarlyStoppingException(Exception):
  pass

def training(model, optimizer, criterion, scheduler, trainX, trainY, valX, valY, epoch, best_val_loss,patience_counter):
  print(f"Epoch {epoch+1} \n ---------- ")
  model.train()
  optimizer.zero_grad()
  outputs = model(trainX)
  outputs = outputs.squeeze(-1)  # Squeeze the output to match the target
  loss = criterion(outputs, trainY)
  loss.backward()
  optimizer.step()

  # Evaluate the model and compute validation loss
  model.eval()
  with torch.no_grad():
      val_outputs = model(valX)
      val_outputs = val_outputs.squeeze(-1)  # Squeeze the output to match the target
      val_loss = criterion(val_outputs, valY)

  # Reduced learning rate when a metric has stopped improving
  scheduler.step(val_loss)
  # Check for early stopping
  if config["training"]["early_stopping"]:
      if val_loss < best_val_loss:
          best_val_loss = val_loss.item()  # Capture the best validation loss
          patience_counter = 0  # Reset counter if validation loss improves
      else:
          patience_counter += 1
          # Stop training if val_loss has not improved after patience limit
          if patience_counter >= config["training"]["early_stopping_patience"]:
              print(f'Early stopping: val_loss has not improved ' +
                    f'for {config["training"]["early_stopping_patience"]} consecutive epochs.')
              raise EarlyStoppingException()


  # Output training/validation statistics
  print(f'Epoch[{epoch+1}/{config["training"]["num_epochs"]}] | ' +
        f'Loss train: {loss.item():.6f}, ' +
        f'Loss val: {val_loss.item():.6f} | ' +
        f'LR: {optimizer.param_groups[0]["lr"]:.6f}'+
       f"best validation loss: {best_val_loss:.6f}" )
  return loss, val_loss, best_val_loss, patience_counter


In [28]:
def testing(model,testX,testY,criterion):
    # Test the model
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        test_outputs = model(testX)
        test_outputs = test_outputs.squeeze(-1)

        test_loss = criterion(test_outputs, testY)
        print(f'Test loss: {test_loss.item():.6f}')
    # compute performance KPY MSE, RMSE, MAE, accuracy, r2
    y_pred = test_outputs.detach().numpy()
    y_true = testY.detach().numpy()
    kpy_mse = mean_squared_error(y_true, y_pred)
    kpy_rmse = np.sqrt(kpy_mse)
    kpy_mae = mean_absolute_error(y_true, y_pred)
    kpy_r2 = r2_score(y_true, y_pred)

    print(f'KPY MSE: {kpy_mse:.6f}, RMSE: {kpy_rmse:.6f}, MAE: {kpy_mae:.6f}, R2: {kpy_r2:.6f}')


    return test_outputs,kpy_mse, kpy_rmse, kpy_mae, kpy_r2


In [29]:
for epoch in range(config["training"]["num_epochs"]):
    try:
        print(f"Epoch {epoch + 1}\n-------------------------------")
        loss, val_loss, best_val_loss, patience_counter = training(model, optimizer, criterion, scheduler, trainX, trainY, valX, valY, epoch, best_val_loss, patience_counter)
        test_outputs, kpy_mse, kpy_rmse, kpy_mae, kpy_r2 = testing(model, testX, testY, criterion)
    except EarlyStoppingException:
        print("Early stopping triggered, proceeding to the next epoch...")
        continue
    print(f"Best validation loss: {best_val_loss:.6f}")
print("Done!")


Epoch 1
-------------------------------
Epoch 1 
 ---------- 
Epoch[1/100] | Loss train: 0.014588, Loss val: 0.026053 | LR: 0.010000best validation loss: 0.003619
Test loss: 0.014385
KPY MSE: 0.014385, RMSE: 0.119936, MAE: 0.114832, R2: -1.514834
Best validation loss: 0.003619
Epoch 2
-------------------------------
Epoch 2 
 ---------- 
Epoch[2/100] | Loss train: 0.022456, Loss val: 0.019779 | LR: 0.010000best validation loss: 0.003619
Test loss: 0.013895
KPY MSE: 0.013895, RMSE: 0.117876, MAE: 0.115275, R2: -1.429164
Best validation loss: 0.003619
Epoch 3
-------------------------------
Epoch 3 
 ---------- 
Epoch[3/100] | Loss train: 0.015218, Loss val: 0.002251 | LR: 0.010000best validation loss: 0.002251
Test loss: 0.002382
KPY MSE: 0.002382, RMSE: 0.048804, MAE: 0.048289, R2: 0.583589
Best validation loss: 0.002251
Epoch 4
-------------------------------
Epoch 4 
 ---------- 
Epoch[4/100] | Loss train: 0.003357, Loss val: 0.005163 | LR: 0.010000best validation loss: 0.002251
Test

In [40]:
# Save the model
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/LSTM_V0.pth")
print(f'ModelLSTMV0 saved to /content/drive/MyDrive/ColabNotebooks')


ModelLSTMV0 saved to /content/drive/MyDrive/ColabNotebooks


In [ ]:
data_predict = test_outputs.data.numpy()
dataY_plot = testY.data.numpy()
# denormalize data
data_predict = scaler.inverse_transform(data_predict)
dataY_plot = scaler.inverse_transform(dataY_plot)
# invert scaling for plotting
dataY_plot = dataY_plot[:, 0]
data_predict = data_predict[:, 0]
# invert scaling for plotting
dataY_plot = dataY_plot[-len(data_predict):]
data_predict = data_predict[-len(data_predict):]
# Plot predictions vs. actual
plt.plot(dataY_plot, label='Data')
plt.plot(data_predict, label='Predictions')
plt.legend()
plt.axvline(x=train_size, c='r', linestyle='--')
plt.suptitle('Time-Series Prediction')
plt.show()

In [ ]:


plt.figure(figsize=(12, 6))
plt.scatter(range(len(all_targets)), all_targets, color='blue', label='Targets', alpha=0.5, marker='o')
plt.scatter(range(len(all_predictions)), all_predictions, color='red', label='Predictions', alpha=0.5, marker='x')

plt.title('Predictions vs Targets')
plt.xlabel('Sample Index')
plt.ylabel('Values')
plt.legend()
plt.show()